In [67]:
import pandas as pd
import numpy as np

## import the data

In [68]:
url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
train = pd.read_csv(url)
train.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Feature Selection

In [69]:
drop_elements = ['PassengerId', 'Name', 'Ticket','Parch','Cabin']
train = train.drop(drop_elements,axis = 1)

# Missing Values

In [70]:
train.isnull().mean() * 100

Survived     0.000000
Pclass       0.000000
Sex          0.000000
Age         19.865320
SibSp        0.000000
Fare         0.000000
Embarked     0.224467
dtype: float64

## Multiple Imputations by Chained Equations(MICE)

In [71]:
string_cols_full = list(train.loc[:, train.dtypes == object].columns[~train.loc[:, train.dtypes == object].isnull().any()])
string_cols_missing_v = list(train.loc[:, train.dtypes == object].columns[train.loc[:, train.dtypes == object].isnull().any()])

#encoding the columns without any missing values
train = pd.get_dummies(train,columns = string_cols_full, drop_first=True)


In [72]:
#from sklearn.impute import SimpleImputer
#imp = SimpleImputer(strategy="most_frequent")
#train['Cabin'] = imp.fit_transform(pd.DataFrame(train['Cabin']))

from fancyimpute import KNN
from sklearn.preprocessing import OrdinalEncoder

#instantiate both packages to use
encoder = OrdinalEncoder()
imputer = KNN()

def encode(data):
    '''function to encode non-null data and replace it in the original data'''
    #retains only non-null values
    nonulls = np.array(data.dropna())
    #reshapes the data for encoding
    impute_reshape = nonulls.reshape(-1,1)
    #encode date
    impute_ordinal = encoder.fit_transform(impute_reshape)
    #Assign back encoded values to non-null values
    data.loc[data.notnull()] = np.squeeze(impute_ordinal)
    return data

#create a for loop to iterate through each column in the data
for columns in string_cols_missing_v:
    encode(train[columns])


/var/folders/39/nd_zbpld0gvbvphjsqgy264w0000gn/T/ipykernel_4622/3365535562.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[data.notnull()] = np.squeeze(impute_ordinal)


In [ ]:
from fancyimpute import IterativeImputer
MICE_imputer = IterativeImputer()
train.iloc[:, :] = MICE_imputer.fit_transform(train)
train = pd.get_dummies(train,columns = string_cols_missing_v, drop_first = True.)

/var/folders/39/nd_zbpld0gvbvphjsqgy264w0000gn/T/ipykernel_4622/2860377829.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[1. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 1. 1. 0. 0. 1. 1. 0. 0. 1. 1. 0. 1.
 0. 0. 1. 1. 0. 1. 1. 0. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0.
 1. 0. 1. 1. 0. 0. 1. 1. 0. 1. 0. 1. 1. 0. 1. 1. 1. 1. 0. 1. 0. 1. 1. 0.
 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 0. 1. 0. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 0. 1. 1. 0. 1. 0. 1. 0. 0. 1. 1. 1. 1. 0.
 1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 0. 0. 1. 1. 0. 1. 1. 1. 0. 0. 0. 1.
 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 0. 0.
 1. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 1. 0. 1. 1. 1. 0. 1. 0. 1. 1. 1. 0. 1.
 0. 1. 0. 0. 1. 1. 0. 0. 1. 1. 1. 1. 1. 0. 1. 1. 0. 1. 1. 0. 1. 1. 1. 0.
 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 0. 1. 0. 1. 0. 1. 1.
 0. 0. 1. 1. 1. 1. 0. 0. 1. 1. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1

In [74]:
train.isnull().mean() * 100

Survived                       0.0
Pclass                         0.0
Age                            0.0
SibSp                          0.0
Fare                           0.0
Sex_male                       0.0
Embarked_0.0                   0.0
Embarked_1.0                   0.0
Embarked_1.2481844244711722    0.0
Embarked_1.3130111960198052    0.0
Embarked_2.0                   0.0
dtype: float64